# Introducción

Crea las series diarias de temperatura máxima, mínima y media de las estaciones de la AEMET en toda España en el periodo 1950-2015.
Los datos provienen de una serie de archivos en las que se dividen por zonas del país y periodos de varios años

In [57]:
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
#from mpl_toolkits.basemap import Basemap, cm
import numpy as np
import glob
import sys
import tqdm
from dateutil import parser
import datetime
from itertools import chain
import csv

In [4]:
# Ruta de los datos y carpetas
ruta_series = 'F:/Series/AEMET/2016_pet080_UNICAN/data/'
folders = ['Ind_0001_3015', 'Ind_3018E_6250', 'Ind_6251_9812E', 'Ind_9813_C939U']
ruta_exportar = 'F:/Series/AEMET/2016_pet080_UNICAN/data/'

# Define el periodo de estudio
time = pd.date_range(start='1950-01-01', end='2015-12-31', freq='D')

### Extraer las estaciones con datos de precipitación

Se extraen las estaciones con alguna medición y sus características (código, nombre, coordenadas, altitud, provincia).

__Lista de archivos__

In [78]:
files = []
for folder in folders:
    # Genera la serie de rutas de los archivos
    f = glob.glob(ruta_series + folder + '/Datos-Pd*')
    f = np.sort(f)
    files.append(f)
    del f
# Convertir la lista a una dimensión
files = list(chain.from_iterable(files))

print('Nº de archivos:', len(files))

Nº de archivos: 61


In [79]:
# Extraer características de las estaciones
estaciones = pd.DataFrame(columns=['INDICATIVO', 'NOMBRE', 'ALTITUD', 'C_X', 'C_Y', 'NOM_PROV', 'LONGITUD', 'LATITUD'])
estaciones.set_index('INDICATIVO', drop=True, inplace=True)

# Buscar estaciones y sus características en todos los archivos
for file in tqdm.tqdm(files):
    aux = pd.read_csv(file, sep=';', encoding='latin-1', usecols=[0, 4, 5, 6, 7, 8, 9 , 10], index_col=0,
                      dtype={'INDICATIVO': str, 'NOM_PROV': str, 'LONGITUD': str})
    for stn in aux.index.unique():
        if stn not in estaciones.index:
            estaciones = pd.concat((estaciones, pd.DataFrame(aux.loc[stn,:].iloc[0]).T), axis=0, join='outer')
            
estaciones.index.name = 'INDICATIVO'
estaciones_pcp = estaciones
#estaciones_pcp['Precipitacion'] = int(1)
del estaciones

print('Nº de estaciones:', estaciones_pcp.shape[0])

100%|██████████████████████████████████████████████████████████████████████████████████| 61/61 [00:45<00:00,  1.35it/s]


Nº de estaciones: 3381


### Extraer las estaciones con datos de precipitación

Se extraen las estaciones con alguna medición y sus características (código, nombre, coordenadas, altitud, provincia).

__Lista de archivos__

In [80]:
files = []
for folder in folders:
    # Genera la serie de rutas de los archivos
    f = glob.glob(ruta_series + folder + '/Datos-Td*')
    f = np.sort(f)
    files.append(f)
    del f
# Convertir la lista a una dimensión
files = list(chain.from_iterable(files))

print('Nº de archivos:', len(files))

Nº de archivos: 25


In [81]:
# Extraer características de las estaciones
estaciones = pd.DataFrame(columns=['INDICATIVO', 'NOMBRE', 'ALTITUD', 'C_X', 'C_Y', 'NOM_PROV', 'LONGITUD', 'LATITUD'])
estaciones.set_index('INDICATIVO', drop=True, inplace=True)

# Buscar estaciones y sus características en todos los archivos
for file in tqdm.tqdm(files):
    aux = pd.read_csv(file, sep=';', encoding='latin-1', usecols=[0, 4, 5, 6, 7, 8, 9 , 10], index_col=0,
                      dtype={'INDICATIVO': str, 'NOM_PROV': str, 'LONGITUD': str})
    for stn in aux.index.unique():
        if stn not in estaciones.index:
            estaciones = pd.concat((estaciones, pd.DataFrame(aux.loc[stn,:].iloc[0]).T), axis=0, join='outer')
    del aux
    
estaciones.index.name = 'INDICATIVO'
estaciones_tmp = estaciones
#estaciones_tmp['Temperatura'] = int(1)
del estaciones

print('Nº de estaciones:', estaciones_tmp.shape[0])

100%|██████████████████████████████████████████████████████████████████████████████████| 25/25 [00:18<00:00,  1.34it/s]


Nº de estaciones: 1627


### Unir las estaciones

In [101]:
estaciones = pd.concat((estaciones_pcp, estaciones_tmp), axis=0, join='outer').drop_duplicates()
estaciones.sort_index(axis=0, ascending=True, inplace=True)

estaciones = estaciones[['NOMBRE', 'NOM_PROV', 'ALTITUD', 'C_X', 'C_Y']]
estaciones.columns = ['NOMBRE', 'PROVINCIA', 'ALTITUD', 'COORD_X', 'COORD_Y']
estaciones['VAR_OBS'] = ''
estaciones.loc[estaciones_pcp.index, 'VAR_OBS'] = estaciones.loc[estaciones_pcp.index, 'VAR_OBS'] + 'Pd'
estaciones.loc[estaciones_tmp.index, 'VAR_OBS'] = estaciones.loc[estaciones_tmp.index, 'VAR_OBS'] + 'Td'
estaciones['DATUM'] = 'ETRS89'

#estaciones['PRECIPITACION'], estaciones['TEMPERATURA'] = 0, 0
#estaciones.loc[estaciones_pcp.index, 'PRECIPITACION'] = 1
#estaciones.loc[estaciones_tmp.index, 'TEMPERATURA'] = 1

estaciones.index = ' ' + estaciones.index
estaciones.index.name = 'INDICATIVO'

# Exportar la tabla de estaciones
estaciones.to_csv(ruta_exportar + 'Estaciones_METEO.txt')

print('Nº total de estaciones:', estaciones.shape[0])
estaciones.head()

Nº total de estaciones: 3386


,NOMBRE,PROVINCIA,ALTITUD,COORD_X,COORD_Y,VAR_OBS,DATUM
INDICATIVO,,,,,,,
0001,EL PERELLO,TARRAGONA,148,812828,4531428,PdTd,ETRS89
0002,L'AMETLLA DE MAR,TARRAGONA,22,820261,4532998,Pd,ETRS89
0002I,VANDELLÓS,TARRAGONA,32,825831,4541326,PdTd,ETRS89
0008,MONTBRIO DEL CAMP,TARRAGONA,120,836154,4559827,PdTd,ETRS89
0009,ALFORJA (HOSPITAL),TARRAGONA,363,833372,4569638,PdTd,ETRS89
